This notebook will address the importance of campaign financing on the share of votes for the primary. This will be done using quantile regressions applied separately on data containing only democratic house of representatives and republican house of
represenatives.
The data prep and cleaning was mostly done along with my project for data 102 class.
However, more markdowns and comments have been added to better show the data preparation process.
1. Data Prep and cleaning

There will be two datasets: Current campaigns and Democratic Candidates. Current Campaigns contains financial campaign data on various candidates. Democratic Candidates contains election data such as proportion of votes along with endorsements from prominent political individuals and groups. The first task is to merge these two datasets in order to regress total receipts from the current campaign dataset on the primary vote column from the democratic candidates data.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
current_campaign = pd.read_csv("Current Campaigns.txt", sep='|', header=None)
correct_headers = [
"CAND_ID", "CAND_NAME", "CAND_ICI", "PTY_CD", "CAND_PTY_AFFILIATION",
"TTL_RECEIPTS", "TRANS_FROM_AUTH", "TTL_DISB", "TRANS_TO_AUTH",
"COH_BOP", "COH_COP", "CAND_CONTRIB", "CAND_LOANS", "OTHER_LOANS",
"CAND_LOAN_REPAY", "OTHER_LOAN_REPAY", "DEBTS_OWED_BY",
"TTL_INDIV_CONTRIB", "CAND_OFFICE_ST", "CAND_OFFICE_DISTRICT",
"SPEC_ELECTION", "PRIM_ELECTION", "RUN_ELECTION", "GEN_ELECTION",
"GEN_ELECTION_PRECENT", "OTHER_POL_CMTE_CONTRIB", "POL_PTY_CONTRIB",
"CVG_END_DT", "INDIV_REFUNDS", "CMTE_REFUNDS"
]
limited_headers = correct_headers[:len(current_campaign.columns)]
current_campaign.columns = limited_headers
current_campaign

,CAND_ID,CAND_NAME,CAND_ICI,PTY_CD,CAND_PTY_AFFILIATION,TTL_RECEIPTS,TRANS_FROM_AUTH,TTL_DISB,TRANS_TO_AUTH,COH_BOP,...,SPEC_ELECTION,PRIM_ELECTION,RUN_ELECTION,GEN_ELECTION,GEN_ELECTION_PRECENT,OTHER_POL_CMTE_CONTRIB,POL_PTY_CONTRIB,CVG_END_DT,INDIV_REFUNDS,CMTE_REFUNDS
0,H8AK00132,"SHEIN, DIMITRI",C,1,DEM,209916.04,0.00,209574.16,0.0,0.00,...,NaN,NaN,NaN,NaN,NaN,0.00,0.0,12/31/2018,0.00,0.0
1,H6AK00045,"YOUNG, DONALD E",I,2,REP,1234680.31,0.00,1387687.05,0.0,269726.86,...,NaN,NaN,NaN,NaN,NaN,559861.90,0.0,12/31/2018,2700.00,500.0
2,H8AK01031,"NELSON, THOMAS JOHN",C,2,REP,9288.48,0.00,8821.97,0.0,0.00,...,NaN,NaN,NaN,NaN,NaN,0.00,0.0,12/31/2018,600.00,0.0
3,H8AK00140,"GALVIN, ALYSE",C,3,IND,1949643.68,154.70,1943398.59,0.0,0.00,...,NaN,NaN,NaN,NaN,NaN,114833.97,0.0,12/31/2018,8166.36,0.0
4,H8AL01066,"KENNEDY, ROBERT JR.",C,1,DEM,166845.21,0.00,166845.21,0.0,0.00,...,NaN,NaN,NaN,NaN,NaN,7750.00,0.0,12/31/2018,0.00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2671,S8WV00150,"BLANKENSHIP, DON",C,3,CON,4165210.09,0.00,4165209.19,0.0,0.00,...,NaN,NaN,NaN,NaN,NaN,0.00,0.0,10/17/2018,0.00,0.0
2672,S8WY00189,"TRAUNER, GARY",C,1,DEM,910722.70,0.00,910723.29,0.0,0.00,...,NaN,NaN,NaN,NaN,NaN,46000.00,0.0,12/07/2018,10572.32,0.0
2673,S4WY00097,"HARDY, CHARLES E",C,2,REP,9058.00,0.00,8891.00,0.0,0.00,...,NaN,NaN,NaN,NaN,NaN,0.00,0.0,12/31/2018,0.00,0.0
2674,S6WY00068,"BARRASSO, JOHN A",I,2,REP,5667279.48,256656.69,4458971.93,0.0,2749487.00,...,NaN,NaN,NaN,NaN,NaN,2555865.00,0.0,12/31/2018,55404.00,47500.0


In [3]:
# Import Democratic Candidate Dataset
dem = pd.read_csv('dem_candidates.csv', encoding='latin1')
# Drop nan values
dem = dem.dropna(subset=["Primary %"])
rep = pd.read_csv('rep_candidates.csv', encoding='latin1')
# Drop nan values
rep = rep.dropna(subset=["Primary %"])
dem_house = dem[dem["Office Type"] == "Representative"]
rep_house = rep[rep["Office Type"] == "Representative"]